# Requirements

## Python

In [1]:
import sys


python_version = (
    f"{sys.version_info.major}."
    f"{sys.version_info.minor}."
    f"{sys.version_info.micro}"
)
print(f"Python version {python_version}")

Python version 3.8.12


Choose any following methods to install dependcies
* poetry (recommend) <br/><br/>
 `poetry install` <br/><br/>
* pip <br/><br/>
`pip install -r requirements.txt`

## IPFS CLI

Follow this URL https://docs.ipfs.io/install/command-line/#official-distributions

# Import modules

In [2]:
import brownie
from brownie import accounts, project, config, network

# Compile smart contracts
p = project.load('.', name="pynft")
p.load_config()

# Connect to chain

In [3]:
import os

print("Available networks")
print(os.popen("brownie networks list").read())

Available networks
Brownie v1.17.0 - Python development framework for Ethereum

The following networks are declared:

Ethereum
  ├─Mainnet (Infura): mainnet
  ├─Ropsten (Infura): ropsten
  ├─Rinkeby (Infura): rinkeby
  ├─Goerli (Infura): goerli
  ├─Kovan (Infura): kovan
  ├─mainnet-fork-2: mainnet-fork-2
  ├─mainnet-fork-3: mainnet-fork-3
  ├─mainnet-fork-4: mainnet-fork-4
  └─mainnet-fork-5: mainnet-fork-5

Ethereum Classic
  ├─Mainnet: etc
  └─Kotti: kotti

Arbitrum
  └─Mainnet: arbitrum-main

Binance Smart Chain
  ├─Testnet: bsc-test
  └─Mainnet: bsc-main

Fantom Opera
  ├─Testnet: ftm-test
  └─Mainnet: ftm-main

Harmony
  └─Mainnet (Shard 0): harmony-main

Polygon
  ├─Mainnet (Infura): polygon-main
  └─Mumbai Testnet (Infura): polygon-test

XDai
  ├─Mainnet: xdai-main
  └─Testnet: xdai-test

Eherium
  └─ganache-local: ganache-local

Etherium
  └─ganache-local-2: ganache-local-2

cronos
  └─cronos-testnet: cronos-testnet

Development
  ├─Ganache-CLI: development
  ├─Geth Dev: geth-d

## Connect

In [4]:
NETWORK = "mainnet"

network.connect(NETWORK)

# Account

In [5]:
!brownie accounts list

Brownie v1.17.0 - Python development framework for Ethereum

Found 4 accounts:
 ├─eth-main: 0xf6EfbD8142A18E741360b41301eDFdbD2719D03C
 ├─prem: 0xedEa45C169753E1af9bf1cb1c374Ca83EBd0d6BE
 ├─test: 0xA3c11aDE2244474EB2F8167921216AB6541F9bA8
 └─test_2: 0xA3c11aDE2244474EB2F8167921216AB6541F9bA8


In [6]:
if NETWORK == "mainnet" and False:
    from scripts.helpful_script import get_account

    account = get_account()

else:
    from brownie import accounts

    accounts.load("eth-main") # Main net
    account = accounts[0]


Enter password for "eth-main": ········


In [7]:
print(account)

0xf6EfbD8142A18E741360b41301eDFdbD2719D03C


# Dump variables

In [8]:
for key, value in p.__dict__.items():
    print(key)
    brownie.__dict__[key] = p.__dict__[key]

_path
_envvars
_structure
_build_path
_name
_active
_sources
_build
_compiler_config
interface
_containers
PYNFT
ERC721
Strings
__all__
_namespaces


# Deploy contracts

In [19]:
from brownie.network import gas_price
from brownie.network.gas.strategies import LinearScalingStrategy
gas_strategy = LinearScalingStrategy("60 gwei", "75 gwei", 1.1)


gas_price(gas_strategy)

In [21]:
from brownie import PYNFT


pynft = PYNFT.deploy({
    "from": account,
    "gas_price": gas_strategy
})

Transaction sent: 0xa5c9b02fe2dfee312bb24fee8ae1bc6edd31339ce7db97a11d40f888a5d89b16
  Gas price: 60.0 gwei   Gas limit: 1419476   Nonce: 11
Transaction sent: 0x2e38964cbe589ae91b5105b87f77e0b44207942a92127cb09ca155649c7f18da
  Gas price: 66.0 gwei   Gas limit: 1419476   Nonce: 11
Transaction sent: 0xd9e803525240d1eb6e4d3e4813a84185451f930e47df4a8c93fdb608cf3950e1
  Gas price: 72.6 gwei   Gas limit: 1419476   Nonce: 11


# Upload file to IPFS

## Assets

In [22]:
prefix = "assets/"


def upload_to_ipfs(filepath: str) -> str:
    return os.popen(f"ipfs add -q {filepath}").read().strip()

def is_an_image(filename: str):
    """
    Description:
        Check if a file is an image
    paramgs:
        filename:
            (str) filename, should be with its extension
    returns:
        (str) qid of ipfs
    """
    for extention in [".png", "jpg"]:
        if filename.endswith(extention):
            return True
    return False

def set_ipfs_share_link(prefix: str, asset: str) -> str:
    """
    description:
        Run IPFS upload process and return share link URL
    params:
        prefix:
            (str) Local prefix file
        asset:
            (str) Filename
    returns:
        (str) share link url
    """
    IPFS_PREFIX = "https://ipfs.io/ipfs"
    qid = upload_to_ipfs(f"{prefix}{asset}".replace(" ", "\ "))
    asset = asset.replace(" ", "-")
    return f"{IPFS_PREFIX}/{qid}?filename={asset}"



class Attribute:
    pass


attribute = Attribute()
for asset in os.listdir(prefix):
    if asset.startswith(".") or asset == "metadata.json":
        # Skip
        continue
    ipfs_share_link = set_ipfs_share_link(prefix, asset)
    if is_an_image(asset):
        setattr(attribute, "image_url", ipfs_share_link)
    else:
        setattr(attribute, "external_url", ipfs_share_link)
    print(ipfs_share_link)

https://ipfs.io/ipfs/Qmda8x41dArU36LKveHzSN3NPJjWxFgyeNNLM8uUMqtMNr?filename=my-first-python-code.png
https://ipfs.io/ipfs/QmUoGY6GbvzSQi2gP7YHtVMRocNW1Xs16ZUHxWcTgWHwJb?filename=fruit-list.py


# Set up metadata

## Render metadata from template

In [23]:
metadata = {
    "description": """It was in 2015, I was an engineer intern.
    
I was assigned to code in Python for satellite signal processing.

At that time, I had no experience with it, but I had completed C#, C and Matlab courses.

It inspired me to transfer from electrical to software engineer career path.

Now, I partially succeed in software engineering, machine learning and A.I implementation.

This NFT determines where I started.

Moreover, I minting this NFT with my Solidity code I created by myself.

No, it's not my first Solidity project :D

But it's my first Solidity project deployed in `production`.

Which means this is the starting point for my smart contract development.

Smart contract repository is located in https://github.com/batprem/pynft
""", 
    "external_url": attribute.external_url, 
    "image": attribute.image_url, 
    "name": "My first Python code",
    "attributes": [], 
}

## Save metadata into JSON

In [24]:
import json


metadata_filename = "metadata.json"

with open(f"assets/{metadata_filename}", "w") as json_file:
    json.dump(metadata, json_file)
    
metadata_share_link = set_ipfs_share_link(prefix, metadata_filename)

# Assign to smart contract

## Mint

In [25]:
PYNFT[-1].createCollectable({"from": account, "gas_limit": 100000}).wait(1)

Transaction sent: 0x2b0ebb107a54e1f4dd8cba236ff325f935a3b8586226002cee6dd7363ba50e7e
  Gas price: 60.0 gwei   Gas limit: 100000   Nonce: 12
Transaction sent: 0x3687db217fd7ba73f5666086b8d193e8afd05ea2441f025ca324f92081b29a3a
  Gas price: 66.0 gwei   Gas limit: 100000   Nonce: 12
Transaction sent: 0xfef11517430a4dec600cde1c9bf9a6d1c1912fab6bdc3800fad85aec3eca93fc
  Gas price: 72.6 gwei   Gas limit: 100000   Nonce: 12


## Set URI

In [29]:
PYNFT[-1].setBaseURI(metadata_share_link, {"from": account, "gas_limit": 120000}).wait(1)

Transaction sent: 0x944d8be4c22c3a0fc2c33403d5909f944f2dd070585571c360f2baab956484a5
  Gas price: 60.0 gwei   Gas limit: 120000   Nonce: 13
Transaction sent: 0x233bdd1900ea80dc6d245ee5f7be85bcbb1fa42f4ff6f328eafe1770dfc8f6ec
  Gas price: 66.0 gwei   Gas limit: 120000   Nonce: 13
This transaction already has 2 confirmations.


In [30]:
PYNFT[-1].tokenURI(0)

'https://ipfs.io/ipfs/QmQBZ6VjMeDphzuTGQxxSst7565yjgwvvs8GHtwbayw6hU?filename=metadata.json'

In [31]:
PYNFT[-1].tokenCounter()

1

# Register to Opensea

In [40]:
from brownie import Contract
import json


with open("utils/open_sea_abi.json", "r") as open_sea_file:
    open_sea_abi = json.load(open_sea_file)


if NETWORK == "mainnet":
    open_sea_address = "0xa5409ec958C83C3f309868babACA7c86DCB077c1"
else:
    raise NotImplementedError("We don't have contract address for this chain yet")
    
OpenSea = Contract.from_abi(name="OpenSea" , address=open_sea_address, abi=open_sea_abi)

In [75]:
gas_strategy = LinearScalingStrategy("60 gwei", "75 gwei", 1.1, time_duration=60)


gas_price(gas_strategy)


OpenSea.registerProxy(
    {"from": account, "gas_price": gas_strategy, "gas_limit": 500000}
)

Transaction sent: 0xae7a0041ba38ae9fc24c97f87ac0efb8e00e7f45f08b88e4fed6fa7ab37806b1
  Gas price: 60.0 gwei   Gas limit: 500000   Nonce: 14
Transaction sent: 0x805ff82ed6ddaa950078ad42b62d077ace37ffc984e8078367dc7fb1eeefb289
  Gas price: 66.0 gwei   Gas limit: 500000   Nonce: 14
Transaction sent: 0x78488589de5dce5b4df59b471004f2324c01388cdcef00db56b18583d1a7537d
  Gas price: 72.6 gwei   Gas limit: 500000   Nonce: 14


Bad pipe message: %s [b'nection: Upgrade\r\nSec-WebSocket-Key: NXiTwOdunv0X6KLCjLo4Hg==\r\nSec-WebSocket-Version: 13\r\nUpgrade: websocket\r\n']


KeyboardInterrupt: 

Sadly, the gas price this time is too high, so I can't register Opensea for now.

## Reference

* https://www.youtube.com/watch?v=M576WGiDBdQ&t=8373s